# Creating a Tool Calling Agent and an Agent Executor

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from langchain import hub

from platform import python_version

from langchain.agents import create_tool_calling_agent, AgentExecutor

In [3]:
wikipedia_tool = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())

In [4]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Introduction-to-Data-and-Data-Science-Course-Lectures", 
                                       description = '''For any questions regarding 
                                       the Introduction to Data and Data Science course, you must use this tool.''')

In [5]:
@tool
def get_python_version() -> str:
    ''' Useful for questions regarding the version of Python currently used. '''
    return python_version()

In [6]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [7]:
chat = ChatOpenAI(model_name = 'gpt-4-0125-preview', 
                  temperature = 0,
                  model_kwargs = {'seed':365})

In [8]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

In [9]:
agent = create_tool_calling_agent(llm = chat, 
                                  tools = tools, 
                                  prompt = chat_prompt_template)

In [10]:
agent_executor = AgentExecutor(agent = agent, 
                               tools = tools, 
                               verbose = True, 
                               return_intermediate_steps = True)

In [11]:
response = agent_executor.invoke({"input":"Could you tell me the version of Python I'm currently using?"})



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.10.10The version of Python you are currently using is 3.10.10.

> Finished chain.


In [12]:
response

{'input': "Could you tell me the version of Python I'm currently using?",
 'output': 'The version of Python you are currently using is 3.10.10.',
 'intermediate_steps': [(ToolAgentAction(tool='get_python_version', tool_input={}, log='\nInvoking: `get_python_version` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_UKqbz9tJB5cvkrVmhAUAS2aU', 'function': {'arguments': '{}', 'name': 'get_python_version'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-97fc1f06-ea0a-4a5b-86da-d98b679b3a42', tool_calls=[{'name': 'get_python_version', 'args': {}, 'id': 'call_UKqbz9tJB5cvkrVmhAUAS2aU'}], tool_call_chunks=[{'name': 'get_python_version', 'args': '{}', 'id': 'call_UKqbz9tJB5cvkrVmhAUAS2aU', 'index': 0}])], tool_call_id='call_UKqbz9tJB5cvkrVmhAUAS2aU'),
   '3.10.10')]}

In [13]:
response = agent_executor.invoke({"input":'''
I’m currently following the Introduction to Data and Data Science course. 
Could you list the programming languages a data scientist should know? 
Additionally, could you tell me who their creators are?
'''})



> Entering new AgentExecutor chain...

Invoking: `Introduction-to-Data-and-Data-Science-Course-Lectures` with `{'query': 'programming languages a data scientist should know'}`


What about big data? Apart from R and Python, people working in this area are often proficient in other languages like Java or Scala. These two have not been developed specifically for doing statistical analyses, however they turn out to be very useful when combining data from multiple sources. All right! Let’s finish off with machine learning. When it comes to machine learning, we often deal with big data

More importantly, it will be sufficient for your need to create quick and accurate analyses. However, if your theoretical preparation is strong enough, you will find yourself restricted by software. Knowing a programming language such as R and Python, gives you the freedom to create specific, ad-hoc tools for each project you are working on

Great! We hope we gave you a good idea about the level of applica

Page: Scala (programming language)
Summary: Scala ( SKAH-lah) is a strong statically typed high-level general-purpose programming language that supports both object-oriented programming and functional programming. Designed to be concise, many of Scala's design decisions are intended to address criticisms of Java.
Scala source code can be compiled to Java bytecode and run on a Java virtual machine (JVM). Scala can also be transpiled to JavaScript to run in a browser, or compiled directly to a native executable. When running on the JVM, Scala provides language interoperability with Java so that libraries written in either language may be referenced directly in Scala or Java code. Like Java, Scala is object-oriented, and uses a syntax termed curly-brace which is similar to the language C. Since Scala 3, there is also an option to use the off-side rule (indenting) to structure blocks, and its use is advised. Martin Odersky has said that this turned out to be the most productive change intr

In [14]:
response

{'input': '\nI’m currently following the Introduction to Data and Data Science course. \nCould you list the programming languages a data scientist should know? \nAdditionally, could you tell me who their creators are?\n',
 'output': 'Based on the Introduction to Data and Data Science course, a data scientist should be proficient in the following programming languages and their creators:\n\n1. **R** - R is a programming language for statistical computing and graphics. The creators of R are Ross Ihaka and Robert Gentleman.\n2. **Python** - Python is a high-level, general-purpose programming language designed for code readability. It was created by Guido van Rossum, who began working on Python in the late 1980s.\n3. **Java** - Java is a high-level, class-based, object-oriented programming language designed to have as few implementation dependencies as possible. It was developed by James Gosling at Sun Microsystems and released in 1995.\n4. **Scala** - Scala is a high-level general-purpose